# The Art of Context Reduction: The Summarizer Agent

Copyright 2025, Denis Rothman

**Goal**:
* Managing API costs and token limits efficiently.    
•	Implementing the count_tokens Utility for proactive cost management.    
•	Building the agent_summarizer to act as an intelligent gatekeeper for large contexts.   
•	Integrating the new agent into the Context Engine's dynamic planning workflow.

**Note on upgrade comments:** The comment *New* in the code is related to the upgrades in made up to and in Chapter 5.

Starting with sthis chapter the files in `commons` will be in new subdirectory of commons. For example for this chapter the `commons` files will be in `commons/chc6/`

**Upgrades in this Chapter described in the book**:

*   **Phase 1**: Verify the `count_tokens` Utility

*   **Phase 2**: Create the New `agent_summarizer` by adding a summarizing agent in the `agents.py` file.

*   **Phase 3**: Register the Summarizer Agent by updating the AgentRegistry in `registry.py`.

*   **Phase 4**: Verify the Core Engine (no changes)

*   **Phase 5**: Run Context Engine
    
    1.Run Control Deck call of Chapter 5 for a backward compatibility check

    2.Run Control Deck example Chapter 6 with new goal and cell














# I. Inititalization

## GitHub

In [ ]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [ ]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/ch6/utils.py")
download_private_github_file("commons/ch6/helpers.py")
download_private_github_file("commons/ch6/agents.py")
download_private_github_file("commons/ch6/registry.py")
download_private_github_file("commons/ch6/engine.py")

✅ Successfully downloaded 'commons/ch6/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/ch6/helpers.py' to './helpers.py'
✅ Successfully downloaded 'commons/ch6/agents.py' to './agents.py'
✅ Successfully downloaded 'commons/ch6/registry.py' to './registry.py'
✅ Successfully downloaded 'commons/ch6/engine.py' to './engine.py'


## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc):
    """
    Runs the context engine with a given goal and configuration,
    then displays the final output and the technical trace.
    """
    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config  # Unpack the config dictionary into keyword arguments
    )

    # 2. Display the Final Result for the main reader
    print("--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace for the developer/technical reader
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # Use pprint for a clean, readable dictionary output
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(trace.steps)

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#II.CONTROL DECK

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


## Control Deck example Chapter 6


In [ ]:
# === CONTROL DECK: Context Reduction Workflow ===

# 1. Define a large piece of text that would be expensive or too long
# to use as direct context for the Writer agent.
large_text_from_researcher = """
Juno is a NASA space probe orbiting the planet Jupiter. It was launched from Cape Canaveral Air Force Station on August 5, 2011, as part of the New Frontiers program. Juno entered a polar orbit of Jupiter on July 5, 2016, to begin a scientific investigation of the planet. After completing its primary mission, it received a mission extension. Juno's mission is to measure Jupiter's composition, gravitational field, magnetic field, and polar magnetosphere. It is also searching for clues about how the planet formed, including whether it has a rocky core, the amount of water present within the deep atmosphere, mass distribution, and its deep winds, which can reach speeds up to 618 kilometers per hour (384 mph). Juno is the second spacecraft to orbit Jupiter, after the nuclear-powered Galileo orbiter, which orbited from 1995 to 2003. Unlike all earlier spacecraft to the outer planets, Juno is powered by solar arrays, which are commonly used by satellites orbiting Earth and working in the inner Solar System, whereas radioisotope thermoelectric generators are commonly used for missions to the outer Solar System and beyond. For Juno, however, the three largest solar array wings ever deployed on a planetary probe play an integral role in stabilizing the spacecraft and generating power.
"""

# 2. Define a goal that requires both using the large text AND a creative step.
# This forces the Planner to recognize the need for summarization before writing.
goal = f"""First, summarize the following text about the Juno probe to extract only the key facts about its scientific mission and instruments. Then, using that summary, write a short, suspenseful scene for a children's story about the probe's dangerous arrival at Jupiter.

--- TEXT TO USE ---
{large_text_from_researcher}
"""

# 3. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 4. Call the execution function
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


I drift toward the giant. Quiet. Focused. My three large solar array wings stretch wide. They hum. They steady me. They drink sun.

Jupiter rises like a shadowed wall. Bands fade and glow. The Great Red Spot rolls by, slow as a breath. The airless dark crackles. I hear it in the radio hiss. The polar magnetosphere sizzles like static.

I aim for the top. A polar orbit. Over the north. Down the south. Around and around.

I listen with metal ears. I measure composition. I feel gravity tug and weigh it. I trace the magnetic field, thread by thread. I sample the polar magnetosphere, bright with auroras.

I hunt for clues. A rocky core, hidden deep. Water, tucked in clouds. Mass distribution, secret and heavy. Deep winds, turning in the dark.

Jupiter waits. Huge. Cold. Its pull is the Source. Its storms whisper. Come closer.

Engines ready. Not yet. Hold. Shadows creep across my decks. The panels creak. Numbers tick. The clock blinks.

Now.

Flame. A steady roar. The craft shivers. Gravity clutches. I do not wobble. My wings keep me true.

Sensors sing. Data pours like rain.

The roar softens. The pull evens. A quiet click.

I slip into a tight polar path.

Orbit. Safe. Triumphant.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 140.20 seconds
Execution Steps:
[ { 'agent': 'Summarizer',
    'output': { 'summary': '- Operates in a polar orbit around Jupiter, '
                           'enabling comprehensive coverage of the poles and '
                           'global mapping.\n'
                           '- Measures Jupiter’s composition, gravitational '
                           'field, magnetic field, and polar magnetosphere.\n'
                           '- Investigates formation by probing the interior '
                           'to assess whether a rocky core exists.\n'
                           '- Determines the amount of water in the deep '
                           'atmosphere to constrain formation models.\n'
                           '- Maps mass distribution and characterizes deep '
                           'atmospheric winds, observed up to about 618 km/h '
                           '(384 mph).\n'
  